# Share Analysis

### Extract data from db and convert it in a seaborn compatible data source

In [ ]:
import pandas as pd
from voices.website import models

query = models.Share.objects.order_by("timestamp").all()

shares= pd.DataFrame({
    "longitude": pd.Series(dtype='float'),
    "latitude": pd.Series(dtype='float'),
    "datetime": pd.Series(dtype="datetime64[ns, UTC]"),
    "timestamp": pd.Series(dtype='int'),
    "message": pd.Series(dtype='str'),
})

for share in models.Share.objects.order_by("timestamp").all():
    x, y = share.mercator_coordinates
    ts = int(share.timestamp.timestamp()) // 60  # in minutes
    shares.loc[len(shares)] = (x, y, share.timestamp, ts, share.message)

shares.timestamp -= shares.timestamp.min()

shares.head()

### Bokeh 

In [ ]:
from bokeh.io import output_notebook, curdoc

output_notebook()
curdoc().theme = 'dark_minimal'

In [ ]:
import bokeh
from bokeh.plotting import figure
from voices.geo.utils import mercator_longitude, mercator_latitude
import xyzservices.providers as xyz

p = figure(
    match_aspect=True, 
    aspect_ratio=2, 
    sizing_mode="stretch_width",
    x_axis_type="mercator", 
    y_axis_type="mercator",
    x_range=(mercator_longitude(10.88), mercator_longitude(10.91)), 
    y_range=(mercator_latitude(44.625), mercator_latitude(44.665)),
    tooltips=[("message", "@message"), ("time", "@timestamp")],
)

if bokeh.__version__ >= "3.0.0":
    p.add_tile(xyz.Stamen.TonerBackground, retina=True)
else:
    from bokeh.tile_providers import CARTODBPOSITRON, get_provider
    tile_provider = get_provider(xyz.Stamen.TonerBackground)
    p.add_tile(tile_provider)
    

In [ ]:
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
from bokeh.transform import linear_cmap
from bokeh.plotting import show

source = ColumnDataSource(shares)

filter_ = BooleanFilter([False] * len(shares))
if bokeh.__version__ >= "3.0.0":
    view = CDSView(filter=filter_)
else:
    view = CDSView(filters=[filter_], source=source)
    
cmap = linear_cmap("timestamp", "Inferno256", low=0, high=shares.timestamp.max())

c = p.circle(
    source=source, 
    view=view,
    x="longitude", 
    y="latitude", 
    color=cmap,
    line_color="gray",
    size=12,
)

handle = show(p, notebook_handle=True)

In [ ]:
import ipywidgets as widgets

t_max = shares.timestamp.max()
ts_slider = widgets.IntRangeSlider(
    value=[0, t_max], min=0, max=t_max, step=1,
    description='Time:'
)

@widgets.interact(value=ts_slider)
def handle_ts_slider_change(value):
    print(value)
    filter_ = BooleanFilter([value[0] <= t <= value[1] for t in shares.timestamp])
    if bokeh.__version__ >= "3.0.0":
        view.filter = filter_
    else:
        view.filters[0] = filter_

    push_notebook(handle=handle)